# Journal Predictor  
  
---------

**Author:** Jose Manuel Gonzalez Fornell

**Email:** josemanuelgonzalezfornell@gmail.com

---------

## Index
1. [Objetive](#1)
2. [Import libraries](#2)
3. [Data Extraction](#3)
4. [Transform data](#4)
5. [Load data](#5)

## Objetive<a id='1'></a>  
It is propose to design an application to predict the journal in which a paper is most probable to be published, using the abstract of the article.

## Import libraries<a id='2'></a>

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../') 
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import re
from sqlalchemy import create_engine, Column, Integer, String, DateTime, text
from sqlalchemy.orm import Session, declarative_base
from sklearn.model_selection import train_test_split
import utils.Extraction_functions as efn
from sklearn.preprocessing import LabelEncoder

## Data Extraction<a id='3'></a>
  
The attack on the Pubmed API is carried out, and the data that provides useful information is obtained in a dataframe. The selected data includes: PMID (index), Article Title, Publication Date, Journal, Journal Abbreviation, Authors, Abstract, Keywords, Locator Format, and Locator Number.

In [2]:
df_final = efn.attack_api_pubmed(email="josemanuelgonzalezfornell@gmail.com",
                                 init_date="2020/01/01", end_date="now", max_results=100, retmax=100, describe=True)

start paper 1/100
finish paper 1/100
start paper 2/100
finish paper 2/100
start paper 3/100
finish paper 3/100
start paper 4/100
finish paper 4/100
start paper 5/100
finish paper 5/100
start paper 6/100
finish paper 6/100
start paper 7/100
finish paper 7/100
start paper 8/100
finish paper 8/100
start paper 9/100
finish paper 9/100
start paper 10/100
finish paper 10/100
start paper 11/100
finish paper 11/100
start paper 12/100
paper 12/100 ignored
start paper 13/100
paper 13/100 ignored
start paper 14/100
finish paper 14/100
start paper 15/100
finish paper 15/100
start paper 16/100
finish paper 16/100
start paper 17/100
finish paper 17/100
start paper 18/100
finish paper 18/100
start paper 19/100
finish paper 19/100
start paper 20/100
finish paper 20/100
start paper 21/100
finish paper 21/100
start paper 22/100
finish paper 22/100
start paper 23/100
finish paper 23/100
start paper 24/100
finish paper 24/100
start paper 25/100
paper 25/100 ignored
start paper 26/100
paper 26/100 ignored


In [3]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52 entries, 38171039 to 38170944
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                52 non-null     object        
 1   Date                 52 non-null     datetime64[ns]
 2   Journal              52 non-null     object        
 3   Journal_abreviation  52 non-null     object        
 4   All_authors          52 non-null     object        
 5   Abstract             52 non-null     object        
 6   Keywords             52 non-null     object        
 7   Locator_format       52 non-null     object        
 8   Locator_number       52 non-null     object        
dtypes: datetime64[ns](1), object(8)
memory usage: 4.1+ KB


## Transform data<a id='4'></a>
  
The data are transformed to have an adequate format. The keywords and authors are transformed into a list of strings separated by commas. The links in the abstract are removed. 

In [4]:
# Format Authors column
df_final["All_authors"] = df_final["All_authors"].apply(
    lambda x: ', '.join(x) if isinstance(x, list) else '')

# Format Keywords column
df_final["Keywords"] = df_final["Keywords"].apply(
    lambda x: ', '.join(x) if isinstance(x, list) else '')

# Format Abstract column
df_final["Abstract"] = df_final["Abstract"].apply(lambda x: re.sub(
    r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', x))

## Load data<a id='5'></a>  
  
The transformed data are loaded into a database using SQLAlchemy.

In [6]:
# Create the database
engine = create_engine(f'sqlite:///../data/processed/Pubmed_DDBB.db')

Base = declarative_base()

# Define the table Main
class Main(Base):
    __tablename__ = 'Main'
    PMID = Column(Integer, primary_key=True, )
    Title = Column(String)
    Date = Column(DateTime)
    Journal = Column(String)
    Journal_abreviation = Column(String)
    All_authors = Column(String)
    Abstract = Column(String)
    Keywords = Column(String)
    Locator_format = Column(String)
    Locator_number = Column(String)

Base.metadata.create_all(engine)
#TODO arreglar para que no se añadan duplicados

# Insert the dataframe into the database
df_final.to_sql('Main', con=engine, index=True, if_exists='replace')
engine = create_engine(f'sqlite:///../data/processed/Pubmed_DDBB.db')
session = Session(engine)
query = text("SELECT * FROM Main")
result = session.execute(query).fetchall()
df_final = df_final.concat(results)
df_final.drop_duplicates(subset="PMID", keep="first", inplace=True)
df_final.to_sql('Main', con=engine, index=False, if_exists='append')

KeyError: Index([38171008, 38171010, 38171011, 38171012, 38171013, 38171016, 38171017,
       38171018, 38171019, 38171020, 38171021, 38171022, 38171023, 38171024,
       38171025, 38171026, 38171029, 38171030, 38171031, 38171032, 38171033,
       38171034, 38171035, 38171036, 38171037, 38171038, 38171039, 38170944,
       38170950, 38170954, 38170955, 38170961, 38170962, 38170963, 38170964,
       38170968, 38170977, 38170979, 38170981, 38170982, 38170984, 38170988,
       38170991, 38170992, 38170993, 38170996, 38170999, 38171001, 38171002,
       38171003, 38171004, 38171007],
      dtype='int64')

# Clean data  
## Stopwords removal

In [25]:
engine = create_engine(f'sqlite:///../data/processed/Pubmed_DDBB.db')
session = Session(engine)

query = text("SELECT PMID, Abstract, Journal FROM Main Limit 100")

result = session.execute(query).fetchall()
df_abstract = pd.DataFrame(result).set_index("PMID")

In [26]:


df_abstract = df_abstract[df_abstract.groupby('Journal')['Journal'].transform('count') > 1]
le = LabelEncoder()
df_abstract['Journal'] = le.fit_transform(df_abstract['Journal'])

In [27]:

X_train, X_test, y_train, y_test = train_test_split(
    df_abstract["Abstract"], df_abstract["Journal"], test_size=0.40, random_state=42, stratify=df_abstract["Journal"])

In [28]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    X_train.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding="longest", 
    truncation=True, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_test.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding="longest", 
    truncation=True, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_test.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [29]:
model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                      num_labels=df_abstract["Journal"].nunique(),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 5

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [31]:

from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [32]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = le.inverse_transform(df_abstract.loc[:, "Journal"])

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [21]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'../data/processed/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([13, 11,  1, 17, 14])
tensor([[  101,  1999,  2344,  ...,     0,     0,     0],
        [  101, 25416, 26884,  ...,  4022,  2004,   102],
        [  101, 22602, 27144,  ...,     0,     0,     0],
        [  101, 20310,  1005,  ...,     0,     0,     0],
        [  101,  1996,  2817,  ...,  8534,  2021,   102]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 4,  7, 12, 11, 18])
tensor([[  101,  2000,  4503,  ...,     0,     0,     0],
        [  101,  5423,  3593,  ...,  1006,  2030,   102],
        [  101,  2312,  2653,  ...,     0,     0,     0],
        [  101, 26180,  2003,  ..., 26180,  1012,   102],
        [  101, 10210, 23924,  ...,     0,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 5, 13,  8,  2,  1])
tensor([[ 101, 2023, 2817,  ...,    0,    0,    0],
        [ 101, 1041, 1011,  ..., 9886, 1997,  102],
        [ 101, 6740, 3853,  ...,    0,    0,    0],
 

Epoch 2:   0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([11,  5, 12,  0, 15])
tensor([[  101, 26180,  2003,  ..., 26180,  1012,   102],
        [  101,  2023,  2817,  ...,     0,     0,     0],
        [  101,  2312,  2653,  ...,     0,     0,     0],
        [  101, 22157,  1011,  ...,     0,     0,     0],
        [  101,  2023,  2817,  ...,   102,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 7,  3,  4,  2, 20])
tensor([[  101,  5423,  3593,  ...,  1006,  2030,   102],
        [  101, 21766,  7629,  ...,     0,     0,     0],
        [  101,  2000,  4503,  ...,     0,     0,     0],
        [  101,  2182,  1010,  ...,     0,     0,     0],
        [  101, 16021,  7934,  ...,     0,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 9, 18, 15, 13, 20])
tensor([[  101, 20415,  6740,  ...,     0,     0,     0],
        [  101, 10210, 23924,  ...,     0,     0,     0],
        [  101,  1996, 16007,  ...,     0

...

torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 2,  8, 15, 14,  6])
tensor([[ 101, 2023, 2817,  ...,    0,    0,    0],
        [ 101, 6740, 3853,  ...,    0,    0,    0],
        [ 101, 2023, 2817,  ...,  102,    0,    0],
        [ 101, 1996, 2817,  ..., 8534, 2021,  102],
        [ 101, 2007, 1996,  ...,    0,    0,    0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 8, 19, 13, 10,  4])
tensor([[  101,  2287,  1011,  ...,     0,     0,     0],
        [  101,  5776,  1011,  ...,     0,     0,     0],
        [  101,  1999,  2344,  ...,     0,     0,     0],
        [  101, 14163, 27108,  ...,     0,     0,     0],
        [  101,  2000,  4503,  ...,     0,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([15, 18,  1,  0, 17])
tensor([[  101,  1996, 16007,  ...,     0,     0,     0],
        [  101,  6887,  7911,  ...,     0,     0,     0],
        [  101,  9253,  4215,  ...,     0,     0,     0],
        [  10

Epoch 4:   0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 4, 14,  6, 19, 16])
tensor([[  101,  2000,  4503,  ...,     0,     0,     0],
        [  101,  1996,  2817,  ...,  8534,  2021,   102],
        [  101,  2007,  1996,  ...,     0,     0,     0],
        [  101,  5776,  1011,  ...,     0,     0,     0],
        [  101,  2195, 23130,  ...,     0,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 2, 10,  7,  0, 20])
tensor([[  101,  2023,  2817,  ...,     0,     0,     0],
        [  101, 14163, 27108,  ...,     0,     0,     0],
        [  101,  5423,  3593,  ...,  1006,  2030,   102],
        [  101, 22157,  1011,  ...,     0,     0,     0],
        [  101,  5372,  4013,  ...,     0,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([18, 12,  3,  1,  2])
tensor([[  101, 10210, 23924,  ...,     0,     0,     0],
        [  101,  2312,  2653,  ...,     0,     0,     0],
        [  101, 21766,  7629,  ...,     0

Epoch 5:   0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 2,  0,  1, 11,  5])
tensor([[  101,  2023,  2817,  ...,     0,     0,     0],
        [  101, 22157,  1011,  ...,     0,     0,     0],
        [  101, 22602, 27144,  ...,     0,     0,     0],
        [  101, 26180,  2003,  ..., 26180,  1012,   102],
        [  101,  2023,  2817,  ...,     0,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 3, 10, 14, 13, 15])
tensor([[  101, 21766,  7629,  ...,     0,     0,     0],
        [  101, 14163, 27108,  ...,     0,     0,     0],
        [  101,  1996,  2817,  ...,  8534,  2021,   102],
        [  101,  1041,  1011,  ...,  9886,  1997,   102],
        [  101,  2023,  2817,  ...,   102,     0,     0]])
torch.Size([5])
torch.Size([5, 512])
torch.Size([5, 512])
tensor([ 2,  6, 12,  5, 18])
tensor([[  101,  1999,  2116,  ...,     0,     0,     0],
        [  101,  2007,  1996,  ...,     0,     0,     0],
        [  101,  2312,  2653,  ...,     0

In [35]:
model = BertForSequenceClassification.from_pretrained("bert-large-uncased",
                                                      num_labels=df_abstract["Journal"].nunique(),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('../data/processed/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Class: JMIR medical education
Accuracy: 2/2

Class: Journal of medical Internet research
Accuracy: 0/1

Class: JMIR formative research
Accuracy: 0/2

Class: JMIR formative research
Accuracy: 0/1

Class: Journal of medical Internet research
Accuracy: 0/1

Class: JMIR medical education
Accuracy: 0/1

Class: JMIR formative research
Accuracy: 0/1

Class: Journal of medical Internet research
Accuracy: 0/1

Class: Plant & cell physiology
Accuracy: 0/1

Class: Plant & cell physiology
Accuracy: 0/1

Class: Plant & cell physiology
Accuracy: 0/1

Class: Plant & cell physiology
Accuracy: 0/1

Class: Current treatment options in oncology
Accuracy: 0/1

Class: Current treatment options in oncology
Accuracy: 0/1

Class: Infection
Accuracy: 0/1

Class: Infection
Accuracy: 1/1

Class: Molecular neurobiology
Accuracy: 0/1

Class: Molecular neurobiology
Accuracy: 0/1

Class: Neurological sciences : official journal of the Italian Neurological Society and of the Italian Society of Clinical Neurophysiolog

In [ ]:
#TODO Eliminar links
